<div align='center' ><font size='70'>Vector</font></div>

## 动态数组 **Vector**

* 存储特性
    * 动态扩容数组
    * 内部维护一段**连续**内存（元素放**满**了怎么办）
    * 本质还是**array**，但**扩容算法**值得学习

* 迭代器
    * 连续内存，拥有 **Random Access Iterator**
    * 迭代器天然有**raw pointer**特性
    * **可能失效**

```c++
    template<typename T, typename Alloc=alloc>
    class vector {
    public:
        typedef T value_type;
        typedef value_type* pointer;
        typedef value_type* iterator;
        typedef value_type& reference;
        typedef size_t size_type;
        typedef ptrdiff_t difference_type;
    
    protected:
        // allocator ...
        iterator start;
        iterator finish;
        iterator end_of_storage;
    
    public:
        // functions ...
    };
```

## vector扩容算法

* 不断 **push_back**，观察 **size**和**capacity** 变化
* [**测试**](./code/vector.cpp)环境
    * **Windows 11 Visual Studio 2022 17.3.6**
    * **WSL 2 Ubuntu 18.04 LTS, GCC 7.5.0** <font color=red>I/O非常慢</font>
<img align="center" src="./src/MSVC-vector.svg" width="85%" border="0">

## [**push_back**](../src/stl_vector.h)实现

* **vector**对外最常见的接口

```c++
    void push_back(const value_type& x) {
        if (finish != end_of_storage) {
            construct(finish, x);
            ++finish;
        }
        else
            insert_aux(end(), x);
    }
```

* 给定迭代器，**insert**可以在**任意位置**插入元素

* 给定迭代器，**erase**可以在**任意位置**删除元素

* **效率**呢？

* 拷贝 **int**也就是**4 bytes**，如果**object**大小是**512bytes**

* **C++11** [**move**](https://en.cppreference.com/w/cpp/utility/move)语义，减少拷贝代价

* 如果只保留**push_back**和**pop_back**，**vector**天然就是一个**stack** （**Adaptor**）

<div align='center' ><font size='70'>Move or Copy</font></div>

## **move**语义对比

* 实验环境
    * **Windows 11 Visual Studio 2022 17.3.6**（带**move**语义）
    * **Windows 11 Visual Studio 2022 SGI STLv3.3**

    <img align="center" src="./src/vector_move_compare.png" width="100%" border="0">

## 具体数据

```c++
    #define LONG_STRING "new looooooooooooooong string"

    int main() {
        constexpr int NUMBER = 10000000;
        std::vector<std::string>bg;
        for (int i = 0; i < NUMBER; ++i) {
            bg.push_back(LONG_STRING);
        }
        std::cout << bg.size() << std::endl;
        for (auto& it : bg) {
            assert(it == LONG_STRING);
        }
        return 0;
}
```

* **x86-release** **10000000**个元素（**assert**不占用时间）
* 时间单位：**秒(s)**
    * **MSVC** **4.19s**
    * **SGI** **51.30s**
* 内存峰值
    * **MSVC** **852MB**
    * **SGI** **1.3GB**
* 热路径
    * **MSVC** **emplace_back**
    * **SGI** **destroy** **operator::new**
* 如果是**int**呢？
    * 两者几乎一致

<div align='center'><font size='70'>End</font></div>